<a href="https://colab.research.google.com/github/GiovanniSorice/Hate_Speech_Detection/blob/main/notebooks/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 ### Removing Tags 

In [216]:
!pip install tokenizer
!pip install ekphrasis
!pip install wordninja
!pip install emoji
!pip install spacy_udpipe
!pip install language_tool_python
!pip install compound-word-splitter
!pip install pyenchant
!sudo apt-get install enchant
!sudo apt-get install hunspell-it

Reading package lists... Done
Building dependency tree       
Reading state information... Done
enchant is already the newest version (1.6.0-11.1).
0 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
hunspell-it is already the newest version (1:6.0.3-3).
0 upgraded, 0 newly installed, 0 to remove and 17 not upgraded.


In [474]:
import pandas as pd
import csv
import nltk
import re
import matplotlib.pyplot as plt
from nltk.tokenize import TweetTokenizer
from tokenizer import *
from nltk.corpus import stopwords

from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

import wordninja

from nltk.stem import SnowballStemmer

import ast

import emoji
import unicodedata

import gzip

import spacy_udpipe
import language_tool_python

import splitter
import enchant
from itertools import groupby
import string

import sys
import os


In [475]:
# directory name:
input_dir = '/content/drive/My Drive/HLT/dataset_training/'
output_dir = '/content/drive/My Drive/HLT/clean_dataset_training/'

# Spec
pd.set_option("display.max_colwidth", None)

In [476]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [477]:
#tsv_tweets_file = open("/content/drive/My Drive/HLT/dataset/haspeede2_reference_taskAB-tweets.tsv")
#tsv_news_file = open("/content/drive/My Drive/HLT/dataset/haspeede2_reference_taskAB-news.tsv")
tsv_file = open(input_dir+"haspeede2_dev_taskAB.tsv")

#dataset_tweets_raw = pd.read_csv(tsv_tweets_file,sep='\t')
#dataset_tweets_raw.rename(columns={"text ": "text"}, inplace=True)
#dataset_news_raw = pd.read_csv(tsv_news_file,sep='\t')
#dataset_news_raw.rename(columns={"text ": "text"}, inplace=True)
dataset_raw = pd.read_csv(tsv_file,sep='\t')
dataset_raw.rename(columns={"text ": "text"}, inplace=True)

# Preprocessing phase
Following the "Preprocessing" section of https://books.openedition.org/aaccademia/4832?lang=it#tocfrom1n4 we will perform:
*   List item
*   List item



### Extraction of the first feature: length of the comment

In [478]:
def comment_length(text):
    return len(text)

In [479]:
dataset_raw['text_length'] = dataset_raw['text'].apply(comment_length)

Extraction of the second feature: percentage of words written in CAPS-LOCK inside the comment.

In [480]:
def caps_lock_words(text):
    words = text.split()
    count_caps_lock = 0
    number_of_words = len(words)
    
    for word in words:
        if word.isupper() == True:
            count_caps_lock = count_caps_lock + 1
            
    return ((count_caps_lock*100)//number_of_words)

In [481]:
dataset_raw['#C-L words'] = dataset_raw['text'].apply(caps_lock_words)

 ### Removing Tags 

In [482]:
def clean_tag(text):
    return re.sub(
        r'@user', ' ', text)

In [483]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_tag)

### Replace the characters ‘&’, ‘@’ respectively in the letters ‘e’, ‘a’

In [485]:
def replace_e_a(text):
    text = re.sub(r'&', 'e', text)
    return re.sub(r'@', 'a', text)

In [486]:
dataset_raw['text'] = dataset_raw['text'].apply(replace_e_a)

### Conversion of disguised bad words

In [487]:
def clean_disguised_bad_words(text):
    text = " " + text + " "
    text = re.sub(r' c[.x*@%#$^]+i ', ' coglioni ', text)
    text = re.sub(r' c[.x*@%#$^]+e ', ' coglione ', text)
    text = re.sub(r' c[.x*@%#$^]+o ', ' cazzo ', text) 
    text = re.sub(r' ca[.x*@%#$^]+ro ', ' cazzaro ', text) 
    text = re.sub(r' c[.x*@%#$^]+i ', ' cazzi ', text) 
    text = re.sub(r' m[.x*@%#$^]+a ', ' merda ', text) 
    text = re.sub(r' m[.x*@%#$^]+e ', ' merde ', text) 
    text = re.sub(r' c[.x*@%#$^]+ulo ', ' culo ', text) 
    text = re.sub(r' p[.x*@%#$^]+a ', ' puttana ', text)
    text = re.sub(r' p[.x*@%#$^]+e ', ' puttane ', text)
    text = re.sub(r' t[.x*@%#$^]+a ', ' troia ', text)
    text = re.sub(r' t[.x*@%#$^]+e ', ' troie ', text)
    text = re.sub(r' s[.x*@%#$^]+o ', ' stronzo ', text)
    text = re.sub(r' s[.x*@%#$^]+i ', ' stronzi ', text)
    return text

In [488]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_disguised_bad_words)

### Hashtag splitting and saving

In [489]:
def find_hashtags(text):
    result = re.findall(r'#\S+', text)
    if result:
        return result
    else:
        return None

In [490]:
dataset_raw['hashtags'] = dataset_raw['text'].apply(find_hashtags)

In [491]:
def split_hashtags(text):
    
    text = ' ' + text + ' '
    result = re.findall(r'#\S+', text)
    
    for word in result:
        new_word = " ".join(splitter.split(word[1:].lower(), 'it_IT'))
        if len(new_word)==0:
            new_word =  word[1:]
                  
        text = text.replace(word, new_word)
        
    return text

In [492]:
dataset_raw['text'] = dataset_raw['text'].apply(split_hashtags)

 ### Removing URL 

In [493]:
def clean_URL(text):
    return re.sub(r'URL', ' ', text)

In [494]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_URL)

### Extraction of the fourth feature: number of ‘?’ or ‘!’ inside the comment.


In [495]:
def esclamations_and_questions(text):
    return text.count('!') + text.count('?')

In [496]:
dataset_raw['#?!'] = dataset_raw['text'].apply(esclamations_and_questions)

### Extraction of the fifth feature: number of ‘.’ or ‘,’ inside the comment.


In [497]:
def esclamations_and_questions(text):
    return text.count(',') + text.count('.')

In [498]:
dataset_raw['#.,'] = dataset_raw['text'].apply(esclamations_and_questions)

### Punctuation removal

In [500]:
def strip_punctuation(text):
    return text.translate(str.maketrans('', '', string.punctuation))

In [501]:
dataset_raw['text'] = dataset_raw['text'].apply(strip_punctuation)

### Removal of nearby equal vowels, removal of nearby equal consonants if they are more than 2

In [503]:
correct_words_vowels = ['sciiti,',
 'welcomerefugees',
 'livetweet',
 'desiree',
 'canaan',
 'tweet.',
 'weekend',
 'romafeyenoord',
 'ebree,',
 'greencard',
 'creerà',
 'cooperative.',
 'moschee,',
 'cooperanti',
 'streetart',
 'khalidmasood',
 'tweet',
 'woolfe',
 'cooperazione',
 'coop',
 'seehofer',
 'speech',
 'coffee',
 'scooter',
 'street',
 'veemenza',
 'moschee.',
 'maalox.',
 'book',
 'tweet',
 'facebook:',
 'sociial,',
 'coop,',
 'canaaniti.',
 'europee,',
 'cooperative',
 'google',
 'creeranno',
 'mediterranee',
 'cooperazione',
 'cooperativa',
 '“boom”',
 'refugees',
 'moonlight',
 'imaam',
 'shooting',
 'sciiti',
 'sunniti',
 'book',
 'atee.',
 'looking',
 'week',
 'ayaan',
 'temporanee.',
 'idee.',
 'sibiliini',
 'food',
 'refugees',
 'retweeted',
 'boom',
 'keep',
 'vodoo',
 'hooligans',
 'ebree',
 'refugees',
 'speed',
 'bloomberg',
 'riina',
 'hatespeech',
 'google',
 'masood',
 'linee.',
 'boom']

In [504]:
def delete_duplicate_vowels_and_redundant_consonant (text):
    parole = text.split()
    stringa = ""
    for a in parole:
        parola = a
        a = [list(g) for k, g in groupby(a)]    
        vocali = ['a','e','i','o','u','y']
        
        for idx,val in enumerate(a):
            if idx == 0:
                stringa += a[idx][0] 
            elif idx == len(a)-1:
                stringa += a[idx][0]
            elif a[idx][0] in vocali and (parola.lower() not in correct_words_vowels):
                stringa += a[idx][0]
            elif len(a[idx]) == 1:
                stringa += a[idx][0]
            elif len (a[idx]) >= 2:
                stringa += a[idx][0]
                stringa += a[idx][1]
        stringa =  stringa + " "
        
    return(stringa)  

In [505]:
dataset_raw['text'] = dataset_raw['text'].apply(delete_duplicate_vowels_and_redundant_consonant)

### Translation of emoticons

In [507]:
def translate_emoticon(text):
    text_result = emoji.demojize(text, language='it')
    text_result=re.sub(r':', ' ', text_result)
    return text_result

In [508]:
dataset_raw['text'] = dataset_raw['text'].apply(translate_emoticon)

In [510]:
emoticons_text = {
    '<kiss>': 'bacio',
    '<happy>': 'felice',
    '<laugh>': 'risata',
    '<sad>': 'triste',
    '<surprise>': 'sorpreso',
    '<wink>': 'occhiolino',
    '<tong>': 'faccia con lingua',
    '<annoyed>': 'annoiato',
    '<seallips>': 'labbra sigillate',
    '<angel>': 'angelo',
    '<devil>': 'diavolo',
    '<highfive>' : 'batti il cinque',
    '<heart>': 'cuore',
    '<user>' : 'persona',
}

In [511]:
def clean_emoticon_text(text):
    text_words = text.split()
    new_words  = [emoticons_text.get(ele, ele) for ele in text_words]
    return ' '.join(new_words)

In [512]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_emoticon_text)

### Replacement of the abbreviations with the respective words

In [468]:
abbr_word = {'cmq':'comunque', 'gov':'governatori', 'fb':'facebook', 'tw':'twitter', 'juve':'juventus', 'ing':'ingegnere', 
             'sx':'sinistra', 'qdo':'quando', 'rep':'repubblica', 'grz':'grazie', 'ita':'italia', 'mln':'milioni', 
             'mld':'miliardi', 'pke':'perche', 'anke':'anche', 'cm':'come', 'dlla':'della', 'dlle':'delle', 'qst':'questa',
             'ke':'che', 'nn':'non', 'sn':'sono', 'cn':'con', 'xk':'perche', 'xke':'perche', 'art':'articolo',
             'tv':'televisore', '€':'euro', 'xché':'perché', 'xké':'perché', 'pkè':'perché'}

In [527]:
def replace_abbreviation(text):
    text_words = text.split()
    new_text = ""
    for token in text_words:
      new_text  += abbr_word.get(token, token) +" "
    
    return new_text.strip()

In [531]:
dataset_raw['text'] = dataset_raw['text'].apply(replace_abbreviation)

### Removal of the laughs

In [532]:
laughs = ['ah', 'eh', 'he' 'ih', 'hi'] #non elimina ahahahah, ma solo ah
vowels = ['a', 'e', 'i', 'o', 'u']

def clean_laughs(text):
    #s = "ahahahah ho fame io, eh eh" -> " ho fame io,"
    text_words = text.split()
    new_words  = [word for word in text_words if word not in laughs]
    
    new_text = ' '.join(new_words)
    
    for i in new_words:
        for k in vowels:
            if ('h' in i) and (len(i) >= 4):
                if (len(i) - 2) <= (i.count(k) + i.count('h')):
                    new_text = new_text.replace(i, '')
    
    return new_text

In [533]:
dataset_raw['text'] = dataset_raw['text'].apply(clean_laughs)

### Tokenization

In [535]:
def tokenization(text):
    tknzr=SocialTokenizer(lowercase=False)
    return tknzr.tokenize(text)

In [536]:
dataset_raw['tokens'] = dataset_raw['text'].apply(tokenization)

In [537]:
dataset_raw.head(30)

,id,text,hs,stereotype,text_length,#C-L words,hashtags,#?!,"#.,",tokens
0,2066,È terrorismo anche questo per mettere in uno stato di soggezione le persone e renderle innocue mentre qualcuno,0,0,120,10,None,0,5,"[È, terrorismo, anche, questo, per, mettere, in, uno, stato, di, soggezione, le, persone, e, renderle, innocue, mentre, qualcuno]"
1,2045,infatti finché ci hanno guadagnato con i campi rom tutto era ok con alemanno ipocriti,0,0,101,0,"[#rom, #Alemanno, #Ipocriti]",0,0,"[infatti, finché, ci, hanno, guadagnato, con, i, campi, rom, tutto, era, ok, con, alemanno, ipocriti]"
2,61,Corriere Tangenti Mafia Capitale dimenticataMazzette su buche e campi rom rom a,0,0,86,8,[#roma],0,1,"[Corriere, Tangenti, Mafia, Capitale, dimenticataMazzette, su, buche, e, campi, rom, rom, a]"
3,1259,ad uno ad uno perché quando i migranti israeliti arrivarono in terra di Canaan fecero fuori tutti i Cananiti,0,0,118,0,None,0,2,"[ad, uno, ad, uno, perché, quando, i, migranti, israeliti, arrivarono, in, terra, di, Canaan, fecero, fuori, tutti, i, Cananiti]"
4,949,Il divertimento del giorno Trovare i patrioti italiani che inneggiano contro i rom facendo la spesa alla li dl multinazionale tedesca,0,0,138,0,[#Lidl],1,1,"[Il, divertimento, del, giorno, Trovare, i, patrioti, italiani, che, inneggiano, contro, i, rom, facendo, la, spesa, alla, li, dl, multinazionale, tedesca]"
5,256,Modena Comune paga la benzina ai nomadi che portano figli a scuola MODENA – La giunta PD…,0,0,96,16,None,0,0,"[Modena, Comune, paga, la, benzina, ai, nomadi, che, portano, figli, a, scuola, MODENA, –, La, giunta, PD, …]"
6,3001,altro che islam o cristianesimo a c dobbiamo sorbire anche dellignorante islam è uno solo,0,1,116,0,None,5,3,"[altro, che, islam, o, cristianesimo, a, c, dobbiamo, sorbire, anche, dellignorante, islam, è, uno, solo]"
7,765,grazie stef stavo giusto caricando ho anche messo Che Salvini avallava il finanziare campi rom,0,0,113,0,None,0,0,"[grazie, stef, stavo, giusto, caricando, ho, anche, messo, Che, Salvini, avallava, il, finanziare, campi, rom]"
8,1157,e smettetela di dire che anche gli italiani sono stati migranti erano trattati male ma non per questo uccidevano innocenti per strada,0,1,140,0,None,0,5,"[e, smettetela, di, dire, che, anche, gli, italiani, sono, stati, migranti, erano, trattati, male, ma, non, per, questo, uccidevano, innocenti, per, strada]"
9,268,Minorenne rom arrestato dopo furto in appartamento,0,1,55,12,None,0,0,"[Minorenne, rom, arrestato, dopo, furto, in, appartamento]"
